In [1]:
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer #see below about "lemmatization"
lemmatizer = WordNetLemmatizer()
import re
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /Users/lan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('Exam_MB210_NLP.csv')

In [3]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
df["description"] = df["tagline"].astype(str) + " " + df["overview"].astype(str)

In [6]:
df["description"].head(1)

0    Enter the World of Pandora. In the 22nd centur...
Name: description, dtype: object

In [7]:
def preprocessing(corpus): #same as previous sessions
    stops = stopwords.words('english') #getting the english stop words list from NLTK
    corpusTokens = [word_tokenize(item) for item in corpus] #tokenizing with the word_tokenize method of NLTK
    def tokenCleaner(token): 
        return lemmatizer.lemmatize(re.sub('[^A-ZÆØÅa-zæøå0-9]','',token.strip().lower()))
    vocabulary = list() #initializing empty vocabulary as list
    for index,item in enumerate(corpusTokens): #looping over each document (which has been tokenized)
        documentTokens = [tokenCleaner(word) 
            for word in item if tokenCleaner(word) and word not in stops]  
        corpusTokens[index] = documentTokens  
        vocabulary += documentTokens  
    corpusNonstop = [(' ').join(document) for document in corpusTokens]  
    return corpusTokens, corpusNonstop, set(vocabulary)  

In [8]:
pd.set_option("display.max_colwidth", None)
df['description'].head(1)

0    Enter the World of Pandora. In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.
Name: description, dtype: object

In [9]:
corpusTokens, corpusNonstop, vocabulary = preprocessing(df.description) 

In [10]:
print(corpusTokens[0],'\n\n',corpusNonstop[0], '\n\n', list(vocabulary)[:20]) #inspecting results

['enter', 'world', 'pandora', 'in', '22nd', 'century', 'paraplegic', 'marine', 'dispatched', 'moon', 'pandora', 'unique', 'mission', 'becomes', 'torn', 'following', 'order', 'protecting', 'alien', 'civilization'] 

 enter world pandora in 22nd century paraplegic marine dispatched moon pandora unique mission becomes torn following order protecting alien civilization 

 ['respective', 'vigilantism', 'arkansas', 'necktie', 'calling', 'warring', 'salondeprovence', 'taliban', 'planet', 'tasker', 'produce', 'campmate', 'impersonal', 'misunderstanding', 'munday', 'psychology', 'minute', 'adoration', 'accursed', 'bolder']


In [11]:
len(list(set(word_tokenize((' ').join(corpusNonstop)))))

21122

In [12]:
df["corpusNonstop"] = corpusNonstop

In [14]:
from collections import Counter
from datetime import datetime

In [15]:
begin_time = datetime.now()


iteration = 0
divide = 1 #variable to vary the fraction of the dataset that is computed, i.e. when testing code

itermax = len(corpusTokens)//divide #the maximum number of iterations, function of 'divide'

count_all = Counter(corpusTokens[0]) #creating a counter, counting for first document

for i in range(1, itermax):
    count_all.update(Counter(corpusTokens[i])) #updating counter object with counts from all other docs.
    iteration+=1

countAllWords = count_all #storing the total counts in a second container (counter object)


#creating a DataFrame of the counter's top 20 words, setting column names and index
count_all = pd.DataFrame(count_all.most_common(20)).rename(columns={0:'word',1:'count'}).set_index('word')


end_time = datetime.now()
print('time spent',end_time-begin_time)
count_all.head(5)

time spent 0:00:00.754813


,count
word,
s,3163
the,1821
a,1678
life,1320
one,1007


In [16]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

# TF-IDF representation on description

TF-IDF: term frequency times inverse document frequency. 

•Term frequencies are the counts of each word in a document

•Inverse document frequency means the inverse document frequency. The number of documents that contain that word goes up, the IDF (and hence the TF-IDF) for that word will go down

In [ ]:
corpusRaw = df.description
corpusLabels = df.title

In [ ]:
def countDoc(document):
    from collections import Counter
    counts = Counter(document)
    return counts

def createBoW_tf(document, vocabulary):
    resultVector = []
    for word in vocabulary:
        if word in document:
            resultVector.append(countDoc(document)[word] / len(document))
        else:
            resultVector.append(0)
    return resultVector

In [ ]:
tfMatrix = []
for document in corpusTokens:
    tfMatrix.append(createBoW_tf(document, vocabulary))
#tfMatrix = np.array(tfMatrix)
tfMatrix = csr_matrix(tfMatrix)
print('shape',tfMatrix.shape)
from random import random
randomItem = int(random()*len(corpusRaw))
print('\nShowing vector for '+corpusLabels[randomItem]+'\n\n',np.round(tfMatrix[randomItem],3))

In [ ]:
#cosine similarity between these three documents, and scaling results between 0 and 1
pairwise_TF = tfMatrix*tfMatrix.T
pairwise_TF = MinMaxScaler().fit_transform(pairwise_TF.toarray().reshape(-1,1)).reshape(len(corpusRaw),len(corpusRaw))

In [ ]:
pd.DataFrame(pairwise_TF,columns=corpusLabels,index=corpusLabels)

In [ ]:
v_tr_tf = TfidfVectorizer(use_idf=True) #initializing the vectorizer
tf = v_tr_tf.fit_transform(corpusNonstop).toarray() #fit, transform corpus
vocabulary_trained = v_tr_tf.vocabulary_.keys() #get vocabulary from corpus

In [ ]:
from scipy.sparse import csr_matrix
tf = csr_matrix(tf)

In [ ]:
pairwise = tf*tf.T

for i in range(pairwise.shape[0]):
    pairwise[i,i] = 0

In [ ]:
source = 2
numberOfMatches = 2

results = np.argsort(pairwise.toarray()[source]).tolist()[-numberOfMatches:]
results.reverse()


print('query:')
print(corpusRaw[source])
print('----------')
print()
print('results:')
print()
for result in results:
    print(corpusRaw[result])
    print('--')

In [ ]:
pd.DataFrame((pairwise).toarray(),columns=corpusLabels,index=corpusLabels) #output pairwise similarity matrix

In [ ]:
corpusLabels

In [ ]:
v_tr_tfidf = TfidfVectorizer(min_df=1, use_idf=True, vocabulary=vocabulary_trained) #using IDF also with Scikit-learn
tfidf = v_tr_tfidf.fit_transform(corpusNonstop) 
pd.DataFrame((tfidf*tfidf.T).toarray(),columns=corpusLabels, index=corpusLabels)

In [ ]:
df[df.title=="Spider-Man"]

In [ ]:
queriesRaw = [ """ 'With great power comes great responsibility. """
               """After being bitten by a genetically altered spider, """
               """nerdy high school student Peter Parker is endowed with amazing powers' """  
             ]

queryLabels = ['Spider-Man']

In [ ]:
queriesNonstop = preprocessing(queriesRaw)[1] #preprocessing the queries

In [ ]:
queriesNonstop

In [ ]:
queriesNonstop = preprocessing(queriesRaw)[1] #preprocessing the queries

tfidfWithQueries = v_tr_tfidf.fit_transform(corpusNonstop)

Spider_Man_Similary_Df = pd.DataFrame((tfidfWithQueries*tfidfWithQueries.T).toarray(), 
             columns=corpusLabels, 
             index=corpusLabels)[queryLabels].drop(queryLabels)

In [ ]:
Spider_Man_Similary_Df.sort_values("Spider-Man", ascending = False).head(10)

# LDiA

•Compute the average position (centroid) of all the TF-IDF vectors 
within the class (such as spam SMS messages).

•Compute the average position (centroid) of all the TF-IDF vectors 
not in the class (such as nonspam SMS messages).

•Compute the vector difference between the centroids (the line 
that connects them).

In [ ]:
# import sms-spam.csv
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf_model.fit_transform(raw_documents=corpusNonstop).toarray()
print(tfidf_docs.shape)
print(type(tfidf_model.fit_transform(raw_documents=corpusNonstop)))

In [ ]:
print(tfidf_model.get_feature_names()[:20])

In [ ]:
print(tfidf_docs[:2])

In [ ]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

n_samples = 4837
n_features = 1000
n_components = 10
n_top_words = 5
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)
tfidf = tfidf_vectorizer.fit_transform(df["corpusNonstop"])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
import pandas as pd
import numpy as np

from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# another implementation with more visualization

lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=5,
    learning_method="online",
    learning_offset=50.0,
    random_state=0,
)

lda.fit(tfidf)

In [ ]:
import matplotlib.pyplot as plt
# plot_top_words, see sklearn documentation: https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    # plt.show()
    plt.savefig('topics.png')



In [ ]:
print(lda.components_)
%matplotlib inline
plot_top_words(lda, tfidf_feature_names, n_top_words, "Topics in LDA model")